In [1]:
import glob
import pandas as pd
import time

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [4]:
%%time
conf = SparkConf()
conf.set("spark.network.timeout", "1000000")
conf.set("spark.executor.heartbeatInterval", "1000000")
conf.set("spark.speculation","false")
conf.set("spark.app.name","blahered")
spark = SparkSession.builder.config(conf = conf).master("local[*]").enableHiveSupport().getOrCreate() #("local[*]") uses all the cpu cores#

Wall time: 7.41 s


In [5]:
spark

In [6]:
sc = spark.sparkContext

In [ ]:
def readfiles(path):
    with open(path,"r",encoding="utf-8") as file:
        text = file.read()
    return text

In [ ]:
%%time
DF = pd.DataFrame(columns=["fileName", "text"])
st_time = time.time()
paths = glob.glob(r'C:\Users\QuadAdmin\Downloads\aclImdb\**\*.txt',recursive = True)

In [ ]:
DF["fileName"] = paths

In [ ]:
DF

In [ ]:
readfiles(DF["fileName"][0])

In [ ]:
read_udf = udf(readfiles, StringType())

In [ ]:
pathsDF = spark.createDataFrame(DF)

In [ ]:
pathsDF.show(5)

In [ ]:
%%time
pathsDF.select("fileName", read_udf("fileName").alias("text")).show(5)

In [ ]:
%%time
pathsDF.select("fileName", read_udf("fileName").alias("text")).write.mode("append").saveAsTable("filesText")

In [ ]:
readDF = spark.read.parquet(r"C:\Users\QuadAdmin\Documents\pyspark_Trials\spark-warehouse\filestext")

In [ ]:
%%time
readDF.show(5)

In [ ]:
from parse_pii.piiparser import *

In [ ]:
classify_udf = udf(classify, StringType())

In [ ]:
%%time
readDF.select("fileName", classify_udf("text").alias("has_pii")).show(5)

In [7]:
gigDF = spark.read.parquet("D:\\tables\\filesMSG_93gig")

In [8]:
gigDF.show(5)

+--------------------+--------------------+
|            fileName|                text|
+--------------------+--------------------+
|C:\Users\QuadAdmi...|Hi Raghu, Thanks ...|
|C:\Users\QuadAdmi...|Hi Raghu, Thanks ...|
|C:\Users\QuadAdmi...|Hi Raghu, Thanks ...|
|C:\Users\QuadAdmi...|Hi Raghu, Thanks ...|
|C:\Users\QuadAdmi...|Hi Raghu, Thanks ...|
+--------------------+--------------------+
only showing top 5 rows

